In [5]:
import os
import sys
import numpy as np
import PIL
from PIL import Image
import torch
# Append custom paths to sys.path for importing custom modules
sys.path.append(os.path.dirname("/home/student/Desktop/31171109-donotdelete/xai-chan/utils"))
from utils.models import ResNet_Model
from utils.transform import resize_transform

# Initialize device
device = torch.device('cuda:0')

In [6]:
# Load model
version = 50
downstream_task_model = ResNet_Model(version=version).to(device)
weights_path = "/home/student/Desktop/31171109-donotdelete/xai-chan/result/imagenet/_Fold_0_5_100X_BreakHis_FT_60_resnet50_imagenet_/_97_94.94949494949495_95.2628090863385_0.9500590562820435.pth"  # TODO: Provide the model path
downstream_task_model.load_state_dict(torch.load(weights_path, map_location=device))
downstream_task_model.eval()

/home/student/anaconda3/envs/xai-chan/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/student/anaconda3/envs/xai-chan/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet_Model(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
       

In [7]:
# Prepare input
# image_path = "/home/student/Desktop/31171109-donotdelete/xai-chan/dataset/benign/SOB/adenosis/SOB_B_A_14-22549AB/100X/SOB_B_A-14-22549AB-100-001.png"  # TODO: Provide the image path
# image_path = "/home/student/Desktop/31171109-donotdelete/xai-chan/dataset/malignant/SOB/mucinous_carcinoma/SOB_M_MC_14-10147/100X/SOB_M_MC-14-10147-100-001.png"
image_path ="/home/student/Desktop/31171109-donotdelete/xai-chan/dataset/benign/SOB/fibroadenoma/SOB_B_F_14-9133/100X/SOB_B_F-14-9133-100-001.png"
input_rgb = Image.open(image_path)
width, height = input_rgb.size
# print(f"Width: {width}, Height: {height}")
input_tensor = resize_transform(np.array(input_rgb)).unsqueeze(0).to(device)

outputs = downstream_task_model(input_tensor)
target_category = (outputs > 0.2).int().item()
print(target_category)

0


In [10]:
from zennit.image import imgify, imsave  # For creating visualizations
from zennit.torchvision import ResNetCanonizer  # For ResNet-specific canonization
from zennit.composites import EpsilonPlusFlat  # For the composite function in LRP
from zennit.attribution import Gradient  # For attributing using gradients
from IPython.display import display
# Make sure your model is in evaluation mode
downstream_task_model.eval()

# Use the ResNet-specific canonizer
canonizer = ResNetCanonizer()

# Create a composite, specifying the canonizers
composite = EpsilonPlusFlat(canonizers=[canonizer])

target = torch.tensor([[1.0]]).to(device)

# Create the attributor, specifying model and composite
with Gradient(model=downstream_task_model, composite=composite) as attributor:
    # Compute the model output and attribution
    output, attribution = attributor(input_tensor, target)

print(f'Prediction: {output.item()}')  # Adapted for a binary classification output

# Sum over the channels
relevance = attribution.sum(1).cpu()

# Create an image of the visualized attribution
img = imgify(relevance, symmetric=True, cmap='coldnhot')



# Convert the image to RGB mode and then display it
img.convert('RGB').show()

# Show the image
# directly save the visualized attribution
imsave('attrib-1.png', relevance, symmetric=True, cmap='coldnhot')

Prediction: 0.00021225518139544874


In [12]:

import torch.nn as nn

import torch

from zennit.attribution import Gradient, SmoothGrad
from zennit.core import Stabilizer
from zennit.composites import EpsilonGammaBox, EpsilonPlusFlat
from zennit.composites import SpecialFirstLayerMapComposite, NameMapComposite

from zennit.rules import Epsilon, ZPlus, ZBox, Norm, Pass, Flat
from zennit.types import Convolution, Activation, AvgPool, Linear as AnyLinear
from zennit.types import BatchNorm, MaxPool
from zennit.torchvision import VGGCanonizer, ResNetCanonizer
# Given that you're working with images in [0, 1] range
low, high = torch.tensor([[[[[0.]]] * 3], [[[[1.]]] * 3]])

composite = SpecialFirstLayerMapComposite(
    layer_map=[
        (nn.ReLU, Pass()),  # if your model uses ReLU activations
        (nn.AvgPool2d, Norm()),  # Normalize relevance for any Average Pooling layers
        (nn.Conv2d, ZPlus()),  # Any convolutional layer
        (nn.Linear, Epsilon(epsilon=1e-6)),  # Linear layers
        (nn.BatchNorm2d, Pass()),  # Ignore BatchNorm layers
    ],
    first_map=[
        (AnyLinear, ZBox(low, high))
    ]
)
target = torch.tensor([[target_category]]).float().to(device)
# directly save the visualized attribution
imsave('attrib-1.png', relevance, symmetric=True, cmap='coldnhot')